In [11]:
import matplotlib
matplotlib.use('Agg')
from pylab import *

from sklearn import linear_model
import cPickle as pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
import pandas as pd
from sklearn import preprocessing
import collections
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

In [2]:
kegg_queries = pickle.load( open( "../data/kegg_queries.p", "rb" ) )

In [3]:
keggs = kegg_queries.keys()
#kegg_lists = [str(k) for k in  kegg_queries.values()]
kegg_lists = [str(kegg_queries[kegg]) for kegg in  keggs]

In [4]:
df = pd.read_csv("../data/keggs_RF_conf_scores.csv")
df.head()

kegg     level_1                  level_2 BRITE_id  \
0  K00001  Metabolism                 Overview      NaN   
1  K00002  Metabolism                 Overview      NaN   
2  K00003  Metabolism                 Overview      NaN   
3  K00004  Metabolism  Carbohydrate metabolism      NaN   
4  K00005  Metabolism  Carbohydrate metabolism      NaN   

                                                   X     LS001     LS002  \
0                      K00001(alcohol_dehydrogenase)  0.005732  0.006140   
1              K00002(alcohol_dehydrogenase_(NADP+))  0.000382  0.000409   
2                   K00003(homoserine_dehydrogenase)  0.000855  0.001034   
3  K00004((R,R)-butanediol_dehydrogenase_/_diacet...  0.000091  0.000106   
4                     K00005(glycerol_dehydrogenase)  0.000167  0.000236   

      LS003     LS004     LS005           ...             HE.SRS017103  \
0  0.006243  0.005962  0.008011           ...                 0.003359   
1  0.000610  0.000533  0.000313           ...                 0.000071   
2  0.001125  0.001074  0.001224           ...                 0.000249   
3  0.000149  0.000116  0.000064           ...                 0.000008   
4  0.000223  0.000203  0.000218           ...                 0.000030   

   HE.SRS017433  HE.SRS018427  HE.SRS018656  HE.SRS020328  HE.SRS020869  \
0      0.004281      0.003052      0.003625      0.002673      0.003999   
1      0.000014      0.000248      0.000649      0.000151      0.000157   
2      0.000131      0.000351      0.000676      0.000430      0.000318   
3      0.000008      0.000014      0.000039      0.000068      0.000010   
4      0.000004      0.000034      0.000036      0.000055      0.000065   

   HE.SRS022713  HE.SRS016585.suspect   over            kegg.selected  
0      0.003734          8.220359e-03  0.542               0 Test set  
1      0.000000          4.812945e-07  0.218               0 Test set  
2      0.000050          1.209999e-06  0.294  1 Used for KS selection  
3      0.000006          4.127393e-04  0.202  1 Used for KS selection  
4      0.000000          2.158647e-04  0.028  1 Used for KS selection  

[5 rows x 70 columns]

In [5]:
kegg_labels = []
bad_keggs = []
for i, kegg in enumerate(keggs):
    if kegg in list(df.kegg):
        kegg_score = df[df.kegg== kegg]["over"].item()
        if kegg_score > 0.75: 
            kegg_labels.append("over")
        elif kegg_score < 0.25:
            kegg_labels.append("under")
        else:
            kegg_labels.append("neither")
    else:
        kegg_labels.append("unknown")
        bad_keggs.append(i)

In [6]:
len(kegg_labels)

10193

In [7]:
len(bad_keggs)

181

In [8]:
le = preprocessing.LabelEncoder()
y_labels = le.fit_transform(kegg_labels)

In [9]:
counter=collections.Counter(y_labels)
counter

Counter({0: 7507, 2: 2057, 1: 448, 3: 181})

In [10]:
counter=collections.Counter(kegg_labels)
counter

Counter({'neither': 7507, 'under': 2057, 'over': 448, 'unknown': 181})

In [12]:
from sklearn import cross_validation
from sklearn.pipeline import Pipeline

In [14]:
text_clf = Pipeline([('vect', CountVectorizer(min_df = 5, analyzer = "word", stop_words="english")),
                     ('tfidf', TfidfTransformer(use_idf=False)),
                     ('sc1', StandardScaler(with_mean=False)),
                     ('clf', SGDClassifier(loss='log', penalty='l2',
                                           alpha=10, n_iter=10, random_state=42))])

In [17]:
parameters = {'clf__alpha': np.logspace(-2, 2, 5)}

In [18]:
good_kegg_list = []
for i, kegg in enumerate(keggs):
    if i in bad_keggs:
        continue
    else:
        good_kegg_list.append(kegg_queries[kegg])

In [19]:
gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1, scoring = "f1_weighted")

In [ ]:
gs_clf.fit(good_kegg_list, delete(y_labels, bad_keggs))

In [20]:
gs_clf.best_score_

AttributeError: 'GridSearchCV' object has no attribute 'best_score_'

In [24]:
gs_clf.best_params_

{'clf__C': 6.1584821106602607}

In [26]:
classificaitons = gs_clf.predict(good_kegg_list)

In [28]:
kegg_ids = []
bad_keggs = []
for i, kegg in enumerate(keggs):
    if kegg in list(df.kegg):
        kegg_ids.append(df[df.kegg== kegg]["kegg"].item())
    else:
        kegg_ids.append("NA")
        bad_keggs.append(i)

good_kegg_list = []
good_keg_ids = []
for i, kegg in enumerate(keggs):
    if i in bad_keggs:
        continue
    else:
        good_keg_ids.append(kegg_ids[i])
        good_kegg_list.append(kegg_lists[i])

In [29]:
classifications_df = pd.DataFrame({'kegg': good_keg_ids, 'classifier_output': classificaitons})
classifications_df.head()

classifier_output    kegg
0                  0  K02040
1                  0  K01369
2                  0  K01368
3                  0  K01365
4                  0  K01364

In [30]:
classifications_df.to_csv("../data/classification-results.csv", index = False)

In [ ]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(good_kegg_list, delete(y_labels, bad_keggs), test_size=0.2, random_state=0)

In [ ]:
text_clf.fit(X_train, y_train)

In [ ]:
text_clf.score(X_train, y_train)

In [ ]:
text_clf.score(X_test, y_test)